1. #### Matlab SPM pipeline for making brain regions and tumor regions in uncorrected, corrected and ground truth DSC space

In [2]:
# Run once to store stdout
import sys
nb_stdout = sys.stdout

In [3]:
# Redirect stdout to console, to not get too much text output in the notebook
# This means that the notebook will not output any text. Text will be redirected
# to the terminal where the notebook was started.
sys.stdout = open(1, 'w')

In [4]:
output_directory_suffix = "2019_07_02_native"
corrections_base_directory = "../epi_corrections_out_" + output_directory_suffix
corrections_base_directory

'../epi_corrections_out_2019_07_02_native'

In [5]:
from os import getcwd
from os.path import abspath
epi_corrections_root_directory = abspath(getcwd() + "/..") # Go one directory up from current working directory
epi_corrections_root_directory

'/media/loek/HDD3TB1/data/IVS_EPI_BASELINE/epi_corrections'

In [11]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_directory" "$output_directory_suffix"
epi_corrections_root_directory=$1
corrections_base_directory=../$2
output_directory_suffix=$3

spm_path=/media/loek/HDD3TB1/apps/spm12 # This needs to be changed
cd $epi_corrections_root_directory
pipeline_report_file=$2/pipeline_report_spm_$output_directory_suffix.txt
run_command='matlab 
                -nodisplay 
                -nosplash 
                -nodesktop 
                -r "cd('"'spm_pipeline'"'); 
                    addpath('"'$spm_path'"'); 
                    myCluster = parcluster; 
                    myCluster.NumWorkers = 4; 
                    saveProfile(myCluster); 
                    parpool(4); 
                    make_dsc_rois('"'$corrections_base_directory'"'); 
                    exit;" 
                2>&1 | tee $pipeline_report_file'
eval $run_command

To see the log, run from corrections_base_directory

tail -f pipeline_report_spm*.txt

2. #### Copy over EPI_raw_DSC, EPI_applytopup and EPI_applyepic to a separate directory

In [3]:
corrections_base_new_directory = \
"../epi_corrections_out_2019_07_02_native_tumor_excluded_from_rcbv"

3. #### Run remove-tumor-from-cbv.ipynb on this separate directory to remove rCBV based on tumor GT-ROIs

4. #### compute-roi-medians.sh using brain and tumor GT ROIs

In [36]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
cd $epi_corrections_root_directory
command="bash scripts/compute-roi-medians.sh \
$corrections_base_new_directory \
2>&1 | tee $corrections_base_new_directory/computegtroismedianslog.txt"
eval $command

5. #### compute-dice-between-rois.sh using brain and tumor GT and uncorrected and corrected ROIs

In [42]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
cd $epi_corrections_root_directory
command="bash scripts/compute-dice-between-rois.sh \
$corrections_base_new_directory \
2>&1 | tee $corrections_base_new_directory/computedicelog.txt"
eval $command

6. #### analyze-medians-and-dice-scores.sh

7. #### analyze-dice-gt-cor.sh

8. #### Additional analysis of GT tumor roi rCBV change from correction (2020-08-31)

See {epi_corrections_root_directory}/notes.txt

In [6]:
corrections_base_new_directory = \
"../epi_corrections_out_2019_07_02_native_wtumor"

In [10]:
print(epi_corrections_root_directory + corrections_base_new_directory)

8. #### a) compute-roi-medians.sh using brain and tumor GT ROIs

In [11]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
cd $epi_corrections_root_directory
command="bash scripts/compute-roi-medians.sh \
$corrections_base_new_directory \
2>&1 | tee $corrections_base_new_directory/computegtroismedianslog.txt"
eval $command

8. #### b) analyze tumor gt median change

In [12]:
corrections_base_old_directory = \
"../epi_corrections_out_2019_07_02_native_tumor_excluded_from_rcbv"

In [78]:
%%bash -s "$epi_corrections_root_directory" "$corrections_base_new_directory"
epi_corrections_root_directory=$1
corrections_base_new_directory=$2
#echo $epi_corrections_root_directory
#echo $corrections_base_new_directory

cd $epi_corrections_root_directory

# Gradient echo (e1) and Spin echo (e2)

for (( i = 1 ; i < 3 ; i++ )) ; do

    # MNI ROI median rCBV files
    readarray epic_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_applyepic -type d -name *e${i}_applyepic_perf | xargs -I {} echo {}/tumorroismedians.txt)

    readarray topup_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_applytopup -type d -name *e${i}_prep_topup_applytopup_postp_perf | xargs -I {} echo {}/tumorroismedians.txt)

    readarray raw_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_raw_DSC -type d -name *e${i}_perf | xargs -I {} echo {}/tumorroismedians.txt)

done

num1=${#epic_tumor_median_files_arr_e1[*]}
#num2=${#topup_tumor_median_files_arr_e1[*]}
#num3=${#raw_tumor_median_files_arr_e1[*]}
#echo $num1
#echo $num2
#echo $num3

script=$epi_corrections_root_directory/scripts/wilcoxon-tumor-gt-rois-analysis.py

command="python $script --rawmedians ${raw_tumor_median_files_arr_e1[@]} --cormedians ${topup_tumor_median_files_arr_e1[@]}"

#echo $command
eval $command

/media/loek/HDD3TB1/data/IVS_EPI_BASELINE/epi_corrections/scripts/wilcoxon-tumor-gt-rois-analysis.py:90: RuntimeWarning: invalid value encountered in less
  top = np.argwhere(np.sort(wilcoxonresults[1, :]) < cutoff)[-1][0] + 1
Traceback (most recent call last):
  File "/media/loek/HDD3TB1/data/IVS_EPI_BASELINE/epi_corrections/scripts/wilcoxon-tumor-gt-rois-analysis.py", line 90, in <module>
    top = np.argwhere(np.sort(wilcoxonresults[1, :]) < cutoff)[-1][0] + 1
IndexError: index -1 is out of bounds for axis 0 with size 0


CalledProcessError: Command 'b'epi_corrections_root_directory=$1\ncorrections_base_new_directory=$2\n#echo $epi_corrections_root_directory\n#echo $corrections_base_new_directory\n\ncd $epi_corrections_root_directory\n\n# Gradient echo (e1) and Spin echo (e2)\n\nfor (( i = 1 ; i < 3 ; i++ )) ; do\n\n    # MNI ROI median rCBV files\n    readarray epic_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_applyepic -type d -name *e${i}_applyepic_perf | xargs -I {} echo {}/tumorroismedians.txt)\n\n    readarray topup_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_applytopup -type d -name *e${i}_prep_topup_applytopup_postp_perf | xargs -I {} echo {}/tumorroismedians.txt)\n\n    readarray raw_tumor_median_files_arr_e${i} < <(find $corrections_base_new_directory/EPI_raw_DSC -type d -name *e${i}_perf | xargs -I {} echo {}/tumorroismedians.txt)\n\ndone\n\nnum1=${#epic_tumor_median_files_arr_e1[*]}\n#num2=${#topup_tumor_median_files_arr_e1[*]}\n#num